calling [`02_Mapping.sh`](./02_Mapping.sh) to map, markdup, sort, and addreadgroups the reads

In [1]:
# set important data in the config file first!
source("config.R")
source("../../extra-R-functions.R")
library(stringdist)

v Reading from Status of mouse sequencing.

v Range ''All founder files''.


Attaching package: 'tidytable'


The following objects are masked from 'package:stats':

    dt, filter, lag


The following object is masked from 'package:base':

    %in%



Attaching package: 'stringdist'


The following object is masked from 'package:tidytable':

    extract




In [2]:
if(!dir.exists(bam_dir)){
    dir.create(bam_dir)
}

In [3]:
head(files_df)

File1,File2,ID,Engram_folder
<chr>,<chr>,<chr>,<chr>
F1_fwd_paired.fq.gz,F1_rev_paired.fq.gz,SW_1,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed
F2_fwd_paired.fq.gz,F2_rev_paired.fq.gz,SW_2,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed
F3_fwd_paired.fq.gz,F3_rev_paired.fq.gz,SW_3,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed
F4_fwd_paired.fq.gz,F4_rev_paired.fq.gz,SW_4,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed
F5_fwd_paired.fq.gz,F5_rev_paired.fq.gz,SW_5,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed
F6_fwd_paired.fq.gz,F6_rev_paired.fq.gz,SW_6,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed


**The script removes the trimmed FASTQ files at the end.**

In [4]:
initial_timedate=Sys.time()

trimmed_files_fwd=list.files(path=trim_dir,pattern="val_1.fq.gz$",full.names = TRUE)
trimmed_files_rev=list.files(path=trim_dir,pattern="val_2.fq.gz$",full.names = TRUE)
for(i in 1:length(trimmed_files_fwd)){
    f1=trimmed_files_fwd[i]
    FileNameDistances=stringdist(a=files_df$File1,b=f1,method = "lv")
    ID=files_df$ID[which(FileNameDistances==min(FileNameDistances))]
    FileNameDistancesRev=stringdist(a=trimmed_files_rev,b=files_df$File2[which(FileNameDistances==min(FileNameDistances))],method = "lv")
    f2=trimmed_files_rev[which(FileNameDistancesRev==min(FileNameDistancesRev))]
    if(file.exists(f2)){
        print(ID)
        print(f1)
        print(f2)
        nthSequenceOfID=which(files_df$File1[files_df$ID==ID]==files_df$File1[which(FileNameDistances==min(FileNameDistances))])
        end_bam=paste0(bam_dir,"/",ID,".",nthSequenceOfID,".bam")
        print(end_bam)
        if(!file.exists(end_bam)){
            cmd=paste0("cd ",bam_dir," && sh -xe ",Barn_Mice_dir,"01_Genotyping/01_Founders/02_Mapping.sh ",
                      f1," ",f2," ",ref_file,
                      " ",ID," ",nthSequenceOfID)
            print(cmd)
            
            # make sbatch and keep a list of all sbatches to be run
            # this allows for drip-fed running of sbatches
            # this is only needed in cases where running everything simulateneouly would be an issue
            # here the issue we want to avoid is the ballooning in size when too many are run at once
            # (SAM files)
            
            sbatch_list=execute_complex_sbatch(list_of_cmds = cmd,jobname = jobname<-"map-founders",
                                               scripts_dir = paste0(bam_dir,"/scripts"), 
                                               uniqueRunID = i,cores = "2",mem = "32G",time = "short",
                                               env = env_mapping_etc, initial_timedate = initial_timedate, 
                                               jobs_simul = 10,list_of_additional_flags=c(paste0("-A ",slurm_acc)),
                                              activateEnvScript=paste0(Barn_Mice_dir,"activateEnv.sh"),
                                              Execute_Sbatches_Env=env_jupyter)
        }
    }
}

[1] "SW_1"
[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/trim_galore/18163FL-04-01-01V_S31_L002_R1_001_val_1.fq.gz"
[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/trim_galore/18163FL-04-01-01V_S31_L002_R2_001_val_2.fq.gz"
[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_1.5.bam"
[1] "SW_2"
[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/trim_galore/18163FL-04-01-02V_S32_L002_R1_001_val_1.fq.gz"
[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/trim_galore/18163FL-04-01-02V_S32_L002_R2_001_val_2.fq.gz"
[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_2.5.bam"
[1] "SW_11"
[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/trim_galore/18163FL-04-01-11V_S41_L002_R1_001_val_1.fq.gz"
[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/trim_galore/18163FL-04-01-11V_S41_L002_R2_001_val_2.fq.gz"
[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_11.5.bam"
[1] "SW_11"
[1] "/moto/ziab/users/jr3950/data/genomes/tmp_founders/t

In [5]:
sbatch_list

ERROR: Error in eval(expr, envir, enclos): object 'sbatch_list' not found


In [6]:
start_sbatch_list(sbatch_list, 10, jobname, initial_timedate)

[1] "2 /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/scripts/map-founders.list"
[1] "[1] \"Currently running 0 jobs.\""               
[2] "[1] \"2 sbatches remaining! Running 11 more...\""
[3] "[1] \"Submitted batch job 13332377\""            
[4] "[1] \"Submitted batch job 13332378\""            
